### Analysis of the Literature Research Process

This notebook contains details about the literature survey

In [ ]:
# First we are going to load the JSON file containing the literature research
import os
import json

current_dir = os.getcwd()
data_dir = os.path.join("./extraction.json")
process_dir = os.path.join("../literature_survey/research_process.json")
references_dir = os.path.join("../literature_survey/research_process.json")

with open(os.path.join(current_dir, data_dir), "r", encoding="utf-8") as f:
    papers_data = json.load(f)

with open(os.path.join(current_dir, process_dir), "r", encoding="utf-8") as f:
    process_data = json.load(f)
    
with open(os.path.join(current_dir, references_dir), "r", encoding="utf-8") as f:
    research_process = json.load(f)

### Number of Question Types by Paper Category and Domain

In [ ]:
from collections import defaultdict
import pandas as pd
# Using defaultdict allows us to avoid checking if a key exists in the dictionary
# it automatically initializes the key with the default value (in this case, 0)
# when a new key is encountered
question_type_counts = defaultdict(int)
paper_counts = defaultdict(int)

for paper in papers_data:
    paper_cat = paper.get("category", "Unknown")
    num_types = len(paper.get("types", []))
    question_type_counts[paper_cat] += num_types
    paper_counts[paper_cat] += 1
data = [
    {
        "Category": cat,
        "Number of Question Types": question_type_counts[cat],
        "Number of Papers": paper_counts[cat]
    }
    for cat in question_type_counts
]
analysis_overall_df = pd.DataFrame(data)

total_question_types = analysis_overall_df["Number of Question Types"].sum()
total_papers = analysis_overall_df["Number of Papers"].sum()

total_row = pd.DataFrame([{
    "Category": "Total",
    "Number of Question Types": total_question_types,
    "Number of Papers": total_papers
}])
analysis_overall_df = pd.concat([analysis_overall_df, total_row], ignore_index=True)

# Sort the DataFrame by Category
analysis_overall_df.sort_values(by=["Category"], inplace=True)
analysis_overall_df.reset_index(drop=True, inplace=True)

print("Analysis: Total number of question types by paper category")
display(analysis_overall_df)

In [ ]:
from collections import defaultdict
import pandas as pd
# Using defaultdict allows us to avoid checking if a key exists in the dictionary
# it automatically initializes the key with the default value (in this case, 0)
# when a new key is encountered
analysis_overall_counts = defaultdict(int)
paper_overall_counts = defaultdict(int)

for paper in papers_data:
    paper_cat = paper.get("category", "Unknown")
    paper_dom = paper.get("domain", "Unknown")
    num_types = len(paper.get("types", []))
    analysis_overall_counts[(paper_cat, paper_dom)] += num_types
    paper_overall_counts[(paper_cat, paper_dom)] += 1

data = [
    {
        "Category": cat,
        "Domain": dom,
        "Number of Question Types": analysis_overall_counts[(cat, dom)],
        "Number of Papers": paper_overall_counts[(cat, dom)]
    }
    for (cat, dom) in analysis_overall_counts
]
analysis_overall_df = pd.DataFrame(data)

# Calculate totals
total_question_types = analysis_overall_df["Number of Question Types"].sum()
total_papers = analysis_overall_df["Number of Papers"].sum()
total_row = pd.DataFrame([{
    "Category": "Total",
    "Domain": "",
    "Number of Question Types": total_question_types,
    "Number of Papers": total_papers
}])

analysis_overall_df = pd.concat([analysis_overall_df, total_row], ignore_index=True)

# Sort the DataFrame by Category and then by Domain
analysis_overall_df.sort_values(by=["Category", "Domain"], inplace=True)
analysis_overall_df.reset_index(drop=True, inplace=True)

print("Analysis: Total number of question types and papers by paper category and domain")
display(analysis_overall_df)

### Publication Year Distribution

In [ ]:
year_counts = defaultdict(int)

for paper in papers_data:
    year = paper.get("year", "Unknown")
    year_counts[year] += 1

data = [
    {"Year": year, "Number of Papers": count}
    for year, count in year_counts.items()
]
analysis_overall_df = pd.DataFrame(data)

analysis_overall_df.sort_values(by=["Year"], inplace=True, ascending=False)
analysis_overall_df.reset_index(drop=True, inplace=True)

print("Analysis: Number of papers published by year")
display(analysis_overall_df)

In [ ]:
# Display as a bar chart and save as PDF
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.bar(analysis_overall_df["Year"], 
    analysis_overall_df["Number of Papers"], 
    color="#79ab74")
plt.xlabel("Year")
plt.ylabel("Number of Papers")
plt.title("Number of Papers Published by Year")
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.yticks(range(0, max(analysis_overall_df["Number of Papers"]) + 1))

plt.savefig("papers_by_year.pdf", bbox_inches='tight')

plt.show()
plt.close()

### Distribution of References
In the following output, the distribution of references is shown. This data reveals the extent to which the classifications may have been informed by prior work.

For this analysis, the "references" fields are used to gather which paper references another paper that is in the final list.

In [ ]:
# First we get the metadata
reference_mapping = {}

for iteration in research_process:
    for paper_process in iteration.get("processing", []):
        title = paper_process.get("title", "No title").lower()        
        if not paper_process.get("added_to_final", False):
            continue
        if reference_mapping.get(title) is None:
                reference_mapping[title] = []
        for reference in paper_process.get("references", []):
            reference_mapping[title].append(reference.lower())

# Now we process each reference_mapping and only keep those references that also have a dict key entry
for title, references in reference_mapping.items():
    delete_idx = []
    for idx, reference in enumerate(references):
        if reference.lower() in reference_mapping.keys():
            continue
        delete_idx.append(idx)
    for idx in reversed(delete_idx):
        del references[idx]


print(f"Total length of references: {len(reference_mapping)}")

rows = []
for idx, (title, references) in enumerate(reference_mapping.items()):
    rows.append({"title": title, "references": references, "amount": len(references)})

df_references = pd.DataFrame(rows)
# sort by amount of references
df_references = df_references.sort_values(by=["amount"], ascending=False)
df_references


### Distribution of Citations
In the following output, the distribution of citations is shown. This provides insight into the relative influence of each publication within this dataset.

For this analysis, the "references" fields are used to gather which paper references another paper that is in the final list. The output is then inverted to show the number of citations per paper.

In [ ]:
# First we get the metadata
citation_mapping = {}
for idx, (title, references) in enumerate(reference_mapping.items()):
    for reference in references:
        if citation_mapping.get(reference) is None:
            citation_mapping[reference] = []
        citation_mapping[reference].append(title)
        
rows = []
for idx, (title, references) in enumerate(citation_mapping.items()):
    rows.append({"title": title, "citations": references, "amount": len(references)})

df_citations = pd.DataFrame(rows)
# sort by amount of citations
df_citations = df_citations.sort_values(by=["amount"], ascending=False)
df_citations